In [ ]:
import json

from dotenv import load_dotenv

load_dotenv()

with open("../data/evaluation/NY_eval_data.json") as f:
    eval_data = json.load(f)

In [ ]:
# dataset creation

from langsmith import Client
from langsmith.utils import LangSmithError

client = Client()
dataset_name = "NYC test"

try:
    # check if dataset exists
    dataset = client.read_dataset(dataset_name=dataset_name)
    print("Since the dataset already exists, we delete and recreate it.")
    client.delete_dataset(
        dataset_id=dataset.id,
    )
except LangSmithError:
    print("No dataset on langsmith.Creating a new dataset.")
finally:
    # if not create a new one with the generated query examples
    dataset = client.create_dataset(dataset_name=dataset_name, description="NYC test dataset")
    for e in eval_data:
        client.create_example(
            inputs={"query": e["question"]},
            outputs={"ground_truths": e["ground_truth"]},
            dataset_id=dataset.id,
        )

    print("Created a new dataset: ", dataset.name)